# Pandapower Optimal Power Flow
This is a program to demonstrate the pandapower optimal power flow function. From the optimal power flow the line loading percentage is sorted out and a constraint is set on a specific line to find out the economic dispatch or the Loacational marginal cost for every buses. Finally a function lmp_for_all_buses is created at the end to find out LMP.

## Example Network

We use the following four bus example network for this exercise:

<img src="Lab_Session_6\example_opf.png" width="50%">

We first create this network in pandapower:

In [328]:
import pandapower as pp
import numpy as np
import math as mt
import sympy as sy
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")
net.trafo.vkr_percent.at[0] = 0
net.trafo.pfe_kw.at[0]=0

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_mw=60, controllable=False)
pp.create_load(net, bus3, p_mw=70, controllable=False)
pp.create_load(net, bus4, p_mw=10, controllable=False)

#create generators
eg = pp.create_ext_grid(net, bus1, min_p_mw=-1000, max_p_mw=1000)
g0 = pp.create_gen(net, bus3, p_mw=80, min_p_mw=0, max_p_mw=80,  vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_mw=100, min_p_mw=0, max_p_mw=100, vm_pu=1.01, controllable=True)

In [329]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_neutral,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service
0,None,100 MVA 220/110 kV,0,1,100.0,220.0,110.0,12.0,0.0,0.0,...,0,-9,9,1.5,0.0,0,False,1,1.0,True


## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization:

In [330]:
costeg = pp.create_poly_cost(net, 0, 'ext_grid', cp1_eur_per_mw=10)
costgen1 = pp.create_poly_cost(net, 0, 'gen', cp1_eur_per_mw=10)
costgen2 = pp.create_poly_cost(net, 1, 'gen', cp1_eur_per_mw=10)

We run an OPF:

In [331]:
pp.runopp(net, delta=1e-16)

This function runs an Optimal Power Flow using the PYPOWER OPF. To make sure that the PYPOWER OPF converges, we decrease the power tolerance `delta` (the default value is `delta=1e-10`). The power tolerance `delta` is a measure of the extent to which exceeding of minimum and maximum power limits is tolerated. That is, in above case, the limits considered by the OPF for the generators are `min_p_mw - delta` and `max_p_mw + delta` as lower and upper bound respectively on the active power. 

Let's check the results:

In [332]:
net.res_ext_grid

,p_mw,q_mvar
0,59.999633,2.55991


In [333]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,69.997330,-1.993949,-4.130836,0.999560
1,10.003033,-1.495470,-4.130649,0.999561


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

## Individual Generator Costs

Let's now assign individual costs to each generator.

We assign a cost of 10 ct/kW for the external grid, 15 ct/kw for the generator g0 and 12 ct/kw for generator g1:

In [334]:
net.poly_cost.cp1_eur_per_mw.at[costeg] = 10
net.poly_cost.cp1_eur_per_mw.at[costgen1] = 15
net.poly_cost.cp1_eur_per_mw.at[costgen2] = 12

And now run an OPF:

In [335]:
pp.runopp(net, delta=1e-16)

We can see that all active power is provided by the external grid: 

In [336]:
net.res_ext_grid

,p_mw,q_mvar
0,143.925799,9.608184


In [337]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000075,8.346090,-16.315952,0.970337
1,0.000205,10.176077,-13.387207,0.992411


This makes sense, because the external grid has the lowest cost of all generators and we did not define any constraints.

The dispatch costs are given in net.res_cost:

In [338]:
net.res_cost

1439.26157143092

# Adding all the equality constraints into the optimum power flow.

In [339]:
def constraints(loading_trafo,loading_line,min_bus_voltage_pu,max_bus_volatge_pu ):

    net.trafo["max_loading_percent"]=loading_trafo
    net.line["max_loading_percent"]=loading_line
    net.bus["min_vm_pu"] = min_bus_voltage_pu
    net.bus["max_vm_pu"] = max_bus_volatge_pu
    
    


In [340]:
constraints(loading_trafo=70,loading_line=70,min_bus_voltage_pu=0.98,max_bus_volatge_pu=1.02)


In [341]:
net.trafo

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_min,tap_max,tap_step_percent,tap_step_degree,tap_pos,tap_phase_shifter,parallel,df,in_service,max_loading_percent
0,None,100 MVA 220/110 kV,0,1,100.0,220.0,110.0,12.0,0.0,0.0,...,-9,9,1.5,0.0,0,False,1,1.0,True,70


In [342]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,max_loading_percent
0,None,149-AL1/24-ST1A 110.0,1,2,70.0,0.194,0.41,8.75,0.0,0.47,1.0,1,ol,True,70
1,None,149-AL1/24-ST1A 110.0,2,3,50.0,0.194,0.41,8.75,0.0,0.47,1.0,1,ol,True,70
2,None,149-AL1/24-ST1A 110.0,3,1,40.0,0.194,0.41,8.75,0.0,0.47,1.0,1,ol,True,70


In [343]:
net.bus

,name,vn_kv,type,zone,in_service,min_vm_pu,max_vm_pu
0,None,220.0,b,None,True,0.98,1.02
1,None,110.0,b,None,True,0.98,1.02
2,None,110.0,b,None,True,0.98,1.02
3,None,110.0,b,None,True,0.98,1.02


We now limit the transformer loading to 70%, line loading to 70% and maximum and minimum bus voltages to 1.02 pu and 0.98 pu.


In [344]:
pp.runopp(net, delta=1e-16)

We can see that the transformer complies with the maximum loading:

In [345]:
net.res_trafo.loading_percent

0    70.000134
Name: loading_percent, dtype: float64

And power generation is now split between the external grid and generator 1 (which is the second cheapest generation unit):

In [346]:
net.res_ext_grid

,p_mw,q_mvar
0,69.981986,-1.419195


In [347]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000021,3.990395,-8.341662,0.983754
1,72.602657,3.494350,-3.790424,1.019834


### Line Loading Constraints

Wen now look at the line loadings:

Now the line loading constraint is complied with:

In [348]:
net.res_line.loading_percent

0    27.894144
1    52.026768
2    17.271836
Name: loading_percent, dtype: float64

And all generators are involved in supplying the loads:

In [349]:
net.res_ext_grid

,p_mw,q_mvar
0,69.981986,-1.419195


In [350]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000021,3.990395,-8.341662,0.983754
1,72.602657,3.494350,-3.790424,1.019834


This of course comes with a once again rising dispatch cost:

In [351]:
net.res_cost

1571.0520605183985

### Voltage Constraints

Finally, we have a look at the bus voltage:

In [352]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.981986,1.419195,10.000000,-1.900697e-22
1,1.005253,-4.792134,60.000000,0.000000,12.211337,2.296098e-02
2,0.983754,-8.341662,69.999979,-3.990395,12.939668,3.328901e-22
3,1.019834,-3.790424,-62.602657,-3.494350,12.000001,3.934404e-22


We can see that all voltages are within the voltage band:

In [353]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar,lam_p,lam_q
0,1.000000,0.000000,-69.981986,1.419195,10.000000,-1.900697e-22
1,1.005253,-4.792134,60.000000,0.000000,12.211337,2.296098e-02
2,0.983754,-8.341662,69.999979,-3.990395,12.939668,3.328901e-22
3,1.019834,-3.790424,-62.602657,-3.494350,12.000001,3.934404e-22


And all generators are once again involved in supplying the loads:

In [354]:
net.res_ext_grid

,p_mw,q_mvar
0,69.981986,-1.419195


In [355]:
net.res_gen

,p_mw,q_mvar,va_degree,vm_pu
0,0.000021,3.990395,-8.341662,0.983754
1,72.602657,3.494350,-3.790424,1.019834


This of course comes once again with rising dispatch costs:

In [356]:
net.res_cost

1571.0520605183985

In [357]:
#Line Impedances
z23 = mt.sqrt((70*0.194)**2 + (70*0.41**2 )) # line impedance from bus 2 to bus 3
z34 = mt.sqrt((50*0.194)**2 + (50*0.41**2 )) # line impedance from bus 3 to bus 4
z24 = mt.sqrt((40*0.194)**2 + (40*0.41**2 )) # line impedance from bus 2 to bus 4
zt = z23 + z24 + z34




# Defining function to find out LMP for all Buses
 Setting constraint on Line 2 (from Bus 2 to Bus 4) - Maximum loading 17%

In [358]:
def lmp_for_all_buses():

    # Setting constrants on Line 2 (from Bus 2 to Bus 4) - Maximum loading 17%
    dp3,dp4 = sy.symbols("dp3 dp4") # dp3 - Power change at Bus 3, dp4 Power change at Bus 4
    
    expr1 = (dp3 + dp4 -1) # Bus3 and Bus 4 to supply 1 MW power to Bus 2.
    
    # for Bus 4 to supply 1 MW to Bus 2, 75% power would flow from bus 4 to Bus 2 and 25% power would flow from Bus 4 to Bus 3 to Bus 2.
    # for Bus 3 to supply 1 MW to Bus 2, 56.25% power would flow from bus 3 to 2 and 43.75% power would flow from Bus 3 to Bus 4 to Bus 2.
    
    expr2 = (((z23+z34)/zt)*dp4 + (z23/zt)*dp3) # Constraint equation
    
    sol = sy.solve((expr1,expr2),(dp3,dp4))
    
    lmp = sol[dp3]*net.poly_cost.cp1_eur_per_mw.at[costgen1] + sol[dp4]*net.poly_cost.cp1_eur_per_mw.at[costgen2]
    print("Loacational marginal cost of Bus 2 is : ", lmp)

    # Setting constrants on Line 2 (from Bus 2 to Bus 4) - Maximum loading 17%
    dp2,dp4 = sy.symbols("dp2 dp4") # dp2 - Power change at Bus 2, dp4 Power change at Bus 4
    
    expr1 = (dp2 + dp4 -1) # Bus 2 and Bus 4 to supply 1 MW power to Bus 3.
    
    # for Bus 2 to supply 1 MW to Bus 56.25% power would flow from Bus 2 to Bus 3 and 43.75% power would flow from Bus 2 to Bus 4 to Bus 3.
    # for Bus 4 to supply 1 MW to Bus 3, 68.75% power would flow from Bus 4 to Bus 3 and 31.25% power would flow from Bus 4 to Bus 2 to Bus 3.
    
    expr2 = (((z24+z23)/zt)*dp2 + (z23/zt)*dp4) # Constraint equation
    
    sol = sy.solve((expr1,expr2),(dp2,dp4))
    
    lmp = sol[dp2]*net.poly_cost.cp1_eur_per_mw.at[costeg] + sol[dp4]*net.poly_cost.cp1_eur_per_mw.at[costgen2]
    print("Loacational marginal cost of Bus 3 is : ", lmp)


    # Setting constrants on Line 2 (from Bus 2 to Bus 4) - Maximum loading 17%
    dp2,dp3 = sy.symbols("dp2 dp3") # dp2 - Power change at Bus 2, dp3 Power change at Bus 3
    
    expr1 = (dp2 + dp3 -1) # Bus 2 and Bus 3 to supply 1 MW power to Bus 4.
    
    # for Bus 2 to supply 1 MW to Bus 4, % power would flow from bus 2 to Bus 4 and % power would flow from Bus 2 to Bus 3 to Bus 4.
    # for Bus 3 to supply 1 MW to Bus 4, % power would flow from bus 3 to 2 and 43.75% power would flow from Bus 3 to Bus 2 to Bus 4.
    
    expr2 = ((z23+z34/zt)*dp2 + (z23/zt)*dp3) # Constraint equation
    
    sol = sy.solve((expr1,expr2),(dp2,dp3))
    
    lmp = sol[dp2]*net.poly_cost.cp1_eur_per_mw.at[costeg] + sol[dp3]*net.poly_cost.cp1_eur_per_mw.at[costgen1]
    print("Loacational marginal cost of Bus 4 is : ", lmp)


In [359]:
lmp_for_all_buses()

Loacational marginal cost of Bus 2 is :  19.1505059738563
Loacational marginal cost of Bus 3 is :  15.4238369924963
Loacational marginal cost of Bus 4 is :  15.1560787134343
